# 1. Tải và tiền xử lý dữ liệu

## Tải dữ liệu

In [1]:
from datasets import load_dataset

ds = load_dataset("conll2003")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/datasets/load.py:1486: FutureWarning: The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [2]:
train_data = ds["train"]
test_data = ds["test"]
valid_data = ds["validation"]

In [3]:
import pandas as pd

train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
valid_df = pd.DataFrame(valid_data)

print(train_df.shape)
print(test_df.shape)
print(valid_df.shape)

(14041, 5)
(3453, 5)
(3250, 5)


In [4]:
train_df

,id,tokens,pos_tags,chunk_tags,ner_tags
0,0,"[EU, rejects, German, call, to, boycott, Briti...","[22, 42, 16, 21, 35, 37, 16, 21, 7]","[11, 21, 11, 12, 21, 22, 11, 12, 0]","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,1,"[Peter, Blackburn]","[22, 22]","[11, 12]","[1, 2]"
2,2,"[BRUSSELS, 1996-08-22]","[22, 11]","[11, 12]","[5, 0]"
3,3,"[The, European, Commission, said, on, Thursday...","[12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 3...","[11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 1...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ..."
4,4,"[Germany, 's, representative, to, the, Europea...","[22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 2...","[11, 11, 12, 13, 11, 12, 12, 11, 12, 12, 12, 1...","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ..."
...,...,...,...,...,...
14036,14036,"[on, Friday, :]","[15, 22, 8]","[13, 11, 0]","[0, 0, 0]"
14037,14037,"[Division, two]","[21, 11]","[11, 12]","[0, 0]"
14038,14038,"[Plymouth, 2, Preston, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]"
14039,14039,"[Division, three]","[21, 11]","[11, 12]","[0, 0]"


## Trích xuất câu và nhãn

In [5]:
train_sentences = ds["train"]["tokens"]
train_tags = ds["train"]["ner_tags"]

test_sentences = ds["test"]["tokens"]
test_tags = ds["test"]["ner_tags"]

valid_sentences = ds["validation"]["tokens"]
valid_tags = ds["validation"]["ner_tags"]

In [6]:
# Lấy label names
ner_label_names = ds["train"].features["ner_tags"].feature.names
print(ner_label_names)

# Mapping id -> label string
id2label = {i: label for i, label in enumerate(ner_label_names)}

def map_ner_tag(example):
    example["ner_tags_str"] = [id2label[label_id] for label_id in example["ner_tags"]]
    return example

# Áp dụng lên toàn bộ tập train
ds = ds.map(map_ner_tag)

print(ds["train"][0]['tokens'])
print(ds["train"][0]['ner_tags'])
print(ds["train"][0]['ner_tags_str'])


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[3, 0, 7, 0, 0, 0, 7, 0, 0]
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


## Xây dựng từ điển

In [7]:
from collections import Counter

def build_vocab(sentences, min_freq=1, lowercase=True):

    word_counter = Counter()
    tag_set = set()

    for sent_tokens, sent_tags in sentences:
        for w, t in zip(sent_tokens, sent_tags):
            if lowercase:
                w = w.lower()
            word_counter[w] += 1
            tag_set.add(t)

    # Tạo word_to_ix, lọc theo min_freq
    word_to_ix = {"<PAD>":0, "<UNK>":1}
    for w, c in word_counter.items():
        if c >= min_freq:
            word_to_ix[w] = len(word_to_ix)

    # Tạo tag_to_ix, thêm <PAD>
    tag_to_ix = {"<PAD>":0}
    for t in sorted(tag_set):
        tag_to_ix[t] = len(tag_to_ix)

    return word_to_ix, tag_to_ix

train_sentences = [
    (example['tokens'], example['ner_tags_str'])
    for example in ds['train']
]

word_to_ix, tag_to_ix = build_vocab(train_sentences, min_freq=1, lowercase=True)

print("Số từ trong vocab:", len(word_to_ix))
print("Các từ đầu tiên:", list(word_to_ix.items())[:10])
print("Các nhãn:", tag_to_ix)

# In kích thước
print("Kích thước từ điển word_to_ix:", len(word_to_ix))
print("Kích thước từ điển tag_to_ix:", len(tag_to_ix))


Số từ trong vocab: 21011
Các từ đầu tiên: [('<PAD>', 0), ('<UNK>', 1), ('eu', 2), ('rejects', 3), ('german', 4), ('call', 5), ('to', 6), ('boycott', 7), ('british', 8), ('lamb', 9)]
Các nhãn: {'<PAD>': 0, 'B-LOC': 1, 'B-MISC': 2, 'B-ORG': 3, 'B-PER': 4, 'I-LOC': 5, 'I-MISC': 6, 'I-ORG': 7, 'I-PER': 8, 'O': 9}
Kích thước từ điển word_to_ix: 21011
Kích thước từ điển tag_to_ix: 10


## Task 2: Tạo PyTorch Dataset và DataLoader

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader

class NERDataset(Dataset):
    def __init__(self, data, word_to_ix, tag_to_ix, max_len=128):
        self.sentences = [example['tokens'] for example in data]
        self.labels = [example['ner_tags_str'] for example in data]
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.sentences[idx]
        tags = self.labels[idx]

        # Encode tokens và tags
        input_ids = [self.word_to_ix.get(t.lower(), self.word_to_ix["<UNK>"]) for t in tokens]
        label_ids = [self.tag_to_ix[t] for t in tags]

        return {"input_ids": input_ids, "labels": label_ids}

In [9]:
from torch.nn.utils.rnn import pad_sequence

# Collate function để padding
def collate_fn(batch):
    input_ids = [torch.tensor(x["input_ids"], dtype=torch.long) for x in batch]
    labels = [torch.tensor(x["labels"], dtype=torch.long) for x in batch]

    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=word_to_ix["<PAD>"])
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=tag_to_ix["<PAD>"])

    return {"input_ids": input_ids, "labels": labels}


In [10]:
# Tạo train & val dataset
train_dataset = NERDataset(ds["train"], word_to_ix, tag_to_ix)
val_dataset = NERDataset(ds["validation"], word_to_ix, tag_to_ix)

# Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

## 3. Xây dựng mô hình RNN

In [11]:
import torch.nn as nn

class SimpleRNNForTokenClassification(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_tags, padding_idx=0):
        super(SimpleRNNForTokenClassification, self).__init__()

        # Embedding đưa token_id -> vector
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=padding_idx
        )

        # RNN xử lý chuỗi
        self.rnn = nn.RNN(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=False
        )

        # Ánh xạ hidden_state -> số lớp nhãn
        self.linear = nn.Linear(hidden_dim, num_tags)

    def forward(self, sentences):
        embeds = self.embedding(sentences)

        rnn_out, _ = self.rnn(embeds)

        tag_scores = self.linear(rnn_out)

        return tag_scores


## Task 4: Huấn luyện mô hình

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# Tham số mô hình
vocab_size = len(word_to_ix)
num_tags = len(tag_to_ix)

embedding_dim = 128
hidden_dim = 256

tag_pad_id = tag_to_ix["<PAD>"]   # ignore_index

# Khởi tạo mô hình
model = SimpleRNNForTokenClassification(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_tags=num_tags,
    padding_idx=word_to_ix["<PAD>"]
).to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Loss: bỏ qua nhãn PAD
criterion = nn.CrossEntropyLoss(ignore_index=tag_pad_id)


In [14]:
def train_model(model, train_loader, optimizer, criterion, num_epochs=5):

    model.train()

    for epoch in range(num_epochs):
        total_loss = 0.0

        for batch in train_loader:

            sentences = batch["input_ids"]
            labels = batch["labels"]

            # Xóa gradient cũ
            optimizer.zero_grad()

            # Forward
            outputs = model(sentences)

            # Reshape
            outputs = outputs.reshape(-1, outputs.size(-1))
            labels = labels.reshape(-1)

            # Tính loss
            loss = criterion(outputs, labels)

            # Backward
            loss.backward()

            # Cập nhật trọng số
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")


# Task 5: Đánh giá mô hình

In [31]:
# !pip install seqeval


In [22]:
# đánh giá (chỉ độ chính xác)
def evaluate_ner(model, val_loader, tag_pad_id):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for batch in val_loader:
            sentences = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(sentences)
            preds = torch.argmax(outputs, dim=-1)

            mask = labels != tag_pad_id  # bỏ padding

            correct += ((preds == labels) & mask).sum().item()
            total += mask.sum().item()

    return correct / total


In [23]:
# Hàm đánh giá đầy đủ

from seqeval.metrics import classification_report

def evaluate_ner_seqeval(model, val_loader, tag_pad_id, id2tag):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            sentences = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(sentences)
            preds = torch.argmax(outputs, dim=-1)

            # Token-level accuracy (bỏ qua padding)
            mask = labels != tag_pad_id
            correct += ((preds == labels) & mask).sum().item()
            total += mask.sum().item()

            # Chuẩn bị dữ liệu cho seqeval
            for pred_seq, label_seq, mask_seq in zip(preds, labels, mask):
                pred_labels, true_labels = [], []
                for p, l, m in zip(pred_seq, label_seq, mask_seq):
                    if m:  # bỏ qua padding
                        pred_labels.append(id2tag[p.item()])
                        true_labels.append(id2tag[l.item()])
                all_preds.append(pred_labels)
                all_labels.append(true_labels)

    accuracy = correct / total
    print(f"Token-level Accuracy: {accuracy*100:.2f}%\n")
    print("Classification Report (seqeval):")
    print(classification_report(all_labels, all_preds))

    return accuracy

## Vòng lặp huấn luyện

In [24]:
num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_loader:
        sentences = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(sentences)

        outputs = outputs.reshape(-1, outputs.size(-1))
        labels = labels.reshape(-1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    val_acc = evaluate_ner(model, val_loader, tag_pad_id)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f} - Val Accuracy: {val_acc*100:.2f}%")


Epoch 1/15 - Loss: 0.0425 - Val Accuracy: 93.61%
Epoch 2/15 - Loss: 0.0303 - Val Accuracy: 93.72%
Epoch 3/15 - Loss: 0.0218 - Val Accuracy: 93.73%
Epoch 4/15 - Loss: 0.0173 - Val Accuracy: 93.69%
Epoch 5/15 - Loss: 0.0152 - Val Accuracy: 93.60%
Epoch 6/15 - Loss: 0.0134 - Val Accuracy: 93.72%
Epoch 7/15 - Loss: 0.0114 - Val Accuracy: 93.81%
Epoch 8/15 - Loss: 0.0106 - Val Accuracy: 93.73%
Epoch 9/15 - Loss: 0.0107 - Val Accuracy: 93.68%
Epoch 10/15 - Loss: 0.0169 - Val Accuracy: 93.63%
Epoch 11/15 - Loss: 0.0146 - Val Accuracy: 93.85%
Epoch 12/15 - Loss: 0.0099 - Val Accuracy: 93.74%
Epoch 13/15 - Loss: 0.0087 - Val Accuracy: 93.86%
Epoch 14/15 - Loss: 0.0087 - Val Accuracy: 93.86%
Epoch 15/15 - Loss: 0.0084 - Val Accuracy: 93.82%


## Dự đoán

In [25]:
# Hàm dự đoán 1 câu
def predict_sentence(model, sentence, word_to_ix, id2tag):
    # Nếu input là string, tách token
    if isinstance(sentence, str):
        tokens = sentence.split()
    else:
        tokens = sentence

    # Chuyển token sang ID
    input_ids = [word_to_ix.get(w.lower(), word_to_ix["<UNK>"]) for w in tokens]
    input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        preds = torch.argmax(outputs, dim=-1).squeeze(0)

    for token, idx in zip(tokens, preds):
        print(f"{token}\t{id2tag[idx.item()]}")


## Kết quả

In [26]:
# Kết quả đánh giá
id2tag = {v: k for k, v in tag_to_ix.items()}
evaluate_ner_seqeval(model, val_loader, tag_pad_id, id2tag)


Token-level Accuracy: 93.82%

Classification Report (seqeval):
              precision    recall  f1-score   support

         LOC       0.86      0.78      0.82      1837
        MISC       0.76      0.65      0.70       922
         ORG       0.64      0.59      0.61      1341
         PER       0.72      0.56      0.63      1842

   micro avg       0.75      0.65      0.70      5942
   macro avg       0.74      0.65      0.69      5942
weighted avg       0.75      0.65      0.69      5942



0.9381838713445738

In [27]:
# Kết quả hàm dự đoán câu mới
sample_sentence = ["Apple", "was", "founded", "by", "Steve", "Jobs", "in", "1976", "."]
predictions = predict_sentence(model, sample_sentence, word_to_ix, id2tag)

Apple	B-ORG
was	O
founded	O
by	O
Steve	B-PER
Jobs	I-PER
in	O
1976	O
.	O


In [30]:
# KẾT QUẢ CUỐI CÙNG
print("KẾT QUẢ THỰC HIỆN")

# Đánh giá trên tập validation
id2tag = {v: k for k, v in tag_to_ix.items()}
final_accuracy = evaluate_ner_seqeval(model, val_loader, tag_pad_id, id2tag)

print("\n" + "="*60)
print(f"Độ chính xác cuối cùng trên tập validation: {final_accuracy*100:.2f}%")


# Dự đoán câu mới
print("\nVí dụ dự đoán câu mới:")
print("-"*60)
sample_sentence = "VNU University is located in Hanoi"
print(f"Câu: {sample_sentence}")
print("Dự đoán:")
predict_sentence(model, sample_sentence, word_to_ix, id2tag)

KẾT QUẢ THỰC HIỆN
Token-level Accuracy: 93.82%

Classification Report (seqeval):
              precision    recall  f1-score   support

         LOC       0.86      0.78      0.82      1837
        MISC       0.76      0.65      0.70       922
         ORG       0.64      0.59      0.61      1341
         PER       0.72      0.56      0.63      1842

   micro avg       0.75      0.65      0.70      5942
   macro avg       0.74      0.65      0.69      5942
weighted avg       0.75      0.65      0.69      5942


Độ chính xác cuối cùng trên tập validation: 93.82%

Ví dụ dự đoán câu mới:
------------------------------------------------------------
Câu: VNU University is located in Hanoi
Dự đoán:
VNU	B-ORG
University	I-ORG
is	O
located	O
in	O
Hanoi	O
